In [271]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import spacy
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [ ]:
def load_data(file):
    with open(file='r', encoding='utf-8') as f:
        data = pd.read_csv(f)
        return (data)
        

In [ ]:
path='/kaggle/input/nlp-getting-started/test.csv'

In [ ]:
data = load_data(path)

In [272]:


df = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
df.head(1)


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1


In [273]:
df.isna().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [274]:
df.shape

(7613, 5)

In [275]:
df = df[['text','target']]

In [276]:
df.iloc[0]["text"]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [ ]:
df.head(5)

In [ ]:
nlp = spacy.load('en_core_web_sm')
#from spacy.lang.en import English
#nlp = English()
#nlp = spacy.blank('en')
#nlp.pipe_names

In [ ]:
# we created a simple Text cat we are adding that to spacy :D 
textcat = nlp.add_pipe('textcat',last=True)

In [ ]:
nlp.pipe_names

In [ ]:
# we created a simple Text cat we are adding that to spacy :D 
textcat = nlp.create_pipe("textcat")

In [ ]:
nlp.pipe_names

In [ ]:
# Adding the labels to textcat
textcat.add_label("POSITIVE")
textcat.add_label("NEGATIVE")

In [ ]:
textcat.labels

In [ ]:
# Converting review text to tuple 
df['tuples'] = df.apply(lambda row: (row['text'],row['target']), axis=1)

In [ ]:
df.head(5)

In [ ]:
# This is how data looks like 
df["tuples"][0]

In [ ]:
# Converting tuple to List 
train = df['tuples'].tolist()

In [ ]:
print(train[4])
#print(len(train))

In [ ]:
texts, labels = zip(*train)

In [ ]:
texts[0]

In [ ]:
labels[0]

In [ ]:
#what we did is bascially created Flags if its Positive aka True and negative we gave False
cats = []
for y in labels:
    if(bool(y)):
        cats.append({"POSITIVE": True, "NEGATIVE":False})
    else:
        cats.append({"POSITIVE": False, "NEGATIVE":True})

In [ ]:
TrainX = texts
TrainY = cats

In [ ]:
len(TrainX)

In [ ]:
len(TrainY)

In [ ]:
#training data has to be in this format
train_data = list(zip(TrainX,[{'cats': cats} for cats in TrainY]))
train_data[0]

In [ ]:
len(train_data)

Model

In [ ]:
n_iter=20

In [ ]:
from thinc.api import Adam

optimizer = Adam(
    learn_rate=0.001,
    beta1=0.9,
    beta2=0.999,
    eps=1e-08,
    L2=1e-6,
    grad_clip=1.0,
    use_averages=True,
    L2_is_weight_decay=True
)

In [ ]:
from spacy.util import minibatch, compounding
from spacy.training.example import Example
# Construction from subclass
from spacy.lang.en import English
nlp = English()

# we created a simple Text cat we are adding that to spacy :D 
textcat = nlp.add_pipe('textcat',last=True)

# Adding the labels to textcat
textcat.add_label("POSITIVE")
textcat.add_label("NEGATIVE")

# Converting review text to tuple 
df['tuples'] = df.apply(lambda row: (row['text'],row['target']), axis=1)

# Converting tuple to List 
train = df['tuples'].tolist()
texts, labels = zip(*train)

#what we did is bascially created Flags if its Positive aka True and negative we gave False
cats = []
for y in labels:
    if(bool(y)):
        cats.append({"POSITIVE": True, "NEGATIVE":False})
    else:
        cats.append({"POSITIVE": False, "NEGATIVE":True})
        
TrainX = texts
TrainY = cats

#training data has to be in this format
train_data = list(zip(TrainX,[{'cats': cats} for cats in TrainY]))

# Disabling other components
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'textcat']
with nlp.disable_pipes(*other_pipes):  # only train textcat
    #optimizer = nlp.begin_training()
    optimizer = optimizer
    print("Training the model...")
    
    # Performing training
    for i in range(len(train_data)):
        print("Epoch : {} ".format(i))
        losses = {}
        batches = minibatch(train_data, size=compounding(4., 32., 1.001))
        for batch in batches:

            texts, annotations = zip(*batch)
                
            example = []
                # Update the model with iterating each text
            for i in range(len(texts)):
                doc = nlp.make_doc(texts[i])
                example.append(Example.from_dict(doc, annotations[i]))
                
                # Update the model
            nlp.update(example, drop=0.5, losses=losses)

In [ ]:
nlp.to_disk("sentiment")

In [ ]:
nlp = spacy.load("sentiment")

In [ ]:
texts[1]

In [ ]:
# Testing the model
test_text = "I had such high hopes for this dress and really crappy worst product hate it wporst bad "
doc=nlp(test_text)
doc.cats 

In [ ]:
texts[0]

In [ ]:
# Testing the model
doc=nlp(texts[1])
doc.cats 